# GraalVM

# Hackathon goals

Begin designing and building web-friendly tools for microscopy data that performantly and interactively render images and volumes and can be easily used in a handful of critical use cases, including desktop applications (e.g. via Electron), web explorers (e.g. the Allen Cell explorer), and within the workflow of computational analysis tools (e.g. CellProfiler, Jupyter). Explore related opportunities around better integrating machine learning into analytical workflows.

## Why care about accessibility?

* Why not ["KISS"](TODO) and make things work _really well_ on one platform (e.g. linux-amd64+python3)?
* Because we want to stop reinventing wheels. That's why we are here planning new JS libraries, right?
* Why do we keep rewriting things?
    * Accessibility - Code does not run on my target platform(s)
    * Licensing - The license is too restrictive
    * Performance - The program is not fast enough
    * Functionality - The program is not powerful enough
    * Quality - The program is brittle, buggy, lacks reproducibility, etc.
    * Complexity - The code is too hard to understand and/or change
    * (Footprint? i.e. "overkill" is this a subset of Complexity?)
    * (Fun. We like coding.)

# Why is Java awesome?

* Write once, run anywhere
* Great performance thanks to the Just-In-Time (JIT) compiler
* Great standard library
* Huge collection of third party libraries
* Awesome dependency management -- great for reproducible science

# Why does Java suck?
* JIT compilation imposes warmup time
* JIT compilation imposes memory overhead
* Java language is verbose
    * Java code is often harder to read and understand than e.g. Python code
    * There are interpreted scripting languages on the JVM (e.g. Jython, JRuby, Groovy) but they suffer performance-wise.

# What is GraalVM?

"One VM to rule them all"—a performant polyglot virtual machine

# What can GraalVM do?

[Top 10 things GraalVM can do](https://github.com/chrisseaton/graalvm-ten-things)

## 1. High-performance modern Java

Graal's Just-In-Time (JIT) compiler is more advanced than OpenJDK's C1 (client) and C2 (server) implementations.
[TODO double check C1 and C2 terms]

* JVMCI [TODO link?] framework enables alternative JIT implementations.

* Written in Java, which means it's significantly easier to hack on and extend than a traditional compiler written in C or C++.

* Advanced optimizations, like partial [escape analysis](https://en.wikipedia.org/wiki/Escape_analysis).

* Twitter now uses it in production with their Scala code for 20% speed boost. Saves lots of money. [TODO link article and/or video]
    * "when applied to Scala programs it runs them about 20% faster." https://blog.plan99.net/graal-truffle-134d8f28fb69

## 2. Low-footprint, fast-startup Java

`native-image`: an Ahead-Of-Time (AOT) compiler
No need to ship the VM.

## 3. Combine JavaScript, Java, Ruby, and R

[Truffle](TODO): a framework for easily and effectively implementing programming languages on the VM.

The "JVM" is not just for Java anymore!

* "That library is not available in my language. I need to rewrite it."
* "That language would be the perfect fit for my problem, but we cannot run it in our environment."
* "That problem is already solved in my language, but the language is too slow."

GraalVM aims "to allow developers to freely choose the right language for the task at hand without making compromises." https://www.graalvm.org/docs/reference-manual/polyglot/

The [Graal Polyglot API](https://www.graalvm.org/docs/reference-manual/polyglot/) lets you [embed and run code from guest languages in JVM-based host applications](http://www.graalvm.org/docs/graalvm-as-a-platform/embed/).

Recognizes interpreters written using Truffle and can convert Truffle abstract syntax trees (ASTs) into optimized native code, using [partial evaluation](https://en.wikipedia.org/wiki/Partial_evaluation).

"invoking a function simply joins the ASTs of the two languages together. Those two ASTs are then compiled and optimized by Graal as a single unit" ([source]())

### How can it possibly work?
- no marshaling/unmarshaling! shared representation of objects

### Which languages
- Java (including JVM-based languages: Clojure, Scala, Groovy, Jython, JRuby, etc.)
    - Caveat: Java itself is not implemented on top of Truffle yet. But [Project Metropolis](TODO) is underway to change that.
- JavaScript
- Ruby
- Python (alpha)
- C, C++, Fortran - via LLVM bitcode

"To give a feel for how easy it is to write these engines, TruffleJS is only about 80,000 lines of code compared to about 1.7 million for V8."
https://blog.plan99.net/graal-truffle-134d8f28fb69

- TruffleRuby ~8x faster than CRuby 2.x:
https://pragtob.wordpress.com/2017/01/24/benchmarking-a-go-ai-in-ruby-cruby-vs-rubinius-vs-jruby-vs-truffle-a-year-later/

## Which languages are implemented?

In [1]:
import org.graalvm.polyglot.Context
context = Context.newBuilder().allowAllAccess(true).build()
context.getEngine().getLanguages()

In [2]:
println('Hello Groovy')
context.eval("js", "print('Hello JavaScript')")
context.eval("python", "print('Hello Python')")
context.eval("R", "print('Hello R');")
context.eval("ruby", "puts('Hello Ruby')")

Hello Groovy
Hello JavaScript
Hello Python
[1] "Hello R"
Hello Ruby


nil

## 4. Run native languages on the JVM

-- node.js native vs jvm mode? Worth clarifying, or no?

## 5. Tools that work across all languages

"The Truffle framework is a kind of nexus for languages and tools."
 -- e.g., Chrome debugger for Python

Debugging
- "Languages such as R and Ruby can be debugged as easily as JavaScript, including stepping through language boundaries during guest language interoperability."
- Chrome DevTools, JVisualVM
- http://www.graalvm.org/docs/reference-manual/tools/
- https://medium.com/graalvm/debugging-polyglot-node-js-ruby-r-apps-with-graalvm-81b1bb2614db

"If you run a standard JVM language, like JRuby, with VisualVM you'll be disappointed in that you'll see the underlying Java objects, rather than any information about your language's objects. If we use the GraalVM version of Ruby instead, VisualVM will recognise the Ruby objects themselves. We need to use the --jvm command to use VisualVM, as it doesn't support the native version of Ruby."

## 6. Extend a JVM-based application

In [3]:
context.eval("python", "[2**n for n in range(0, 16)]")

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]

## 7. Extend a native application
- link to libpolyglot and invoke any/all Truffle languages from C et al

## 8. Java code as a native library

TODO

## 9. Polyglot in the database

TODO

## 10. Create your own language

Truffle languages are written in Java.

"An advanced visualizer tool lets you explore the compiler's intermediate representation as it passes through optimization stages." https://blog.plan99.net/graal-truffle-134d8f28fb69

# Questions

## What are the downsides?
Space/time tradeoffs: warmup time, memory usage, vs. static comp
* JIT = more memory, longer startup and warmup times
* AOT = less memory, fast startup, but less dynamic/extensible and often worse peak performance

## Will it catch on?
Graal's improved JIT already ships with Java 9 and 10.
```
java -XX:+UnlockExperimentalVMOptions -XX:+EnableJVMCI -XX:+UseJVMCICompiler -Djvmci.Compiler=graal ...
```
~30 core developers at Oracle Labs and academic partners.

## But... it's Oracle! What about open source?

TODO - break down the licensing of components

I am impressed with not only the open licensing, but also the openness of process. Academic papers and partnerships. All on GitHub. Active chat room on Gitter.

## What about Python?

org.graalvm.python implementation still alpha quality:

[TODO: demo of it sucking]

But it is a development priority of the core team (~30-40 devs).

In the meantime, we can still invoke Java from Python with shared memory using e.g. pyjnius.
Combined with Graal, this is very powerful: Python + &lt;any-Truffle-language&gt; for free.

## What about PyPy?

[TODO mention TruffleRuby's C blending approach?]
"Because scripting languages are so slow it's very common to rewrite performance hotspots in dynamically typed programs by hand in C, using the original interpreter's internal API to interact with the scripted code. Perversely, this technique actually makes it harder to speed up the language in general because running real programs often means running their C extensions too, and that's very difficult when those extensions make so many assumptions about the runtime's internals."
https://blog.plan99.net/graal-truffle-134d8f28fb69

## What about JS in the browser?
JavaScript in the browser is tied to specific JS implementations (Chrome = V8, Firefox = SpiderMonkey). Unclear (to me) if GraalVM's JS implementation is usable in browser... but seems unlikely.
To maximize space and time performance, we still need interprocess shared memory between the browser's JS engine and the backend. But at least now the backend is easier to unify.

## What about Electron?
Graal team says it is feasible to build Electron to use graal's node.js instead. But no examples yet.
[TODO quote from Gitter]

## What about mobile?
[TODO]

## Shared memory

In [ ]:
bigArray = new double[300 * 1000 * 1000]

In [5]:
// We can push a variable directly into a language's global scope.
context.getBindings("js").putMember("bigArray", bigArray)
context.eval("js", "print('JavaScript: array length = ' + bigArray.length)")

JavaScript: array length = 300000000


undefined

In [6]:
// We can use the polyglot bindings to avoid polluting the language's global namespace.
context.getPolyglotBindings().putMember("bigArray", bigArray)
context.eval("python",
"""
import polyglot
big_array = polyglot.import_value('bigArray')
print('Python: array length = ' + str(len(big_array)))
"""
)

Python: array length = 300000000


None